# Spectral Axes: Velocity Frames and Doppler Conventions

This notebook shows how to change spectral axes to represent different rest frames and Doppler conventions.  In addition, a spectrum is normally
associated with a redshift/radial velocity  (PJT?)

Supported doppler conventions:
1. radio
2. relativistic
3. optical

Supported rest frames (some of)
1. ITRF or topocentric
2. ICRF or heliocentric
3. GCRS
4. LSRK, LSRD
5. cmb

This dysh code dealing with this relies heavily on [specutils](https://specutils.readthedocs.io/)

Although the `Spectrum.plot()` can plot a spectrum in different frames with different conventions, it does not modify the underlying data and meta-data in the spectrum.  To make these persistent (useful when writing a spectrum) there is both an in-place and copy operation to modify frame and convention. Here's a dysh command summary that we will cover in this notebook, leaving out the values and arguments that are not relevant:

```
       ta.plot(vel_frame=, doppler_convention=)

       ta.velocity_axis_to(toframe=, doppler_convention=)

       ta.set_frame()
       ta.set_convention()

       ta1 = ta.with_frame()
       ta2 = ta1.with_velocity_convention()

       ta.set_redshift.to()
       ta.set_radial_velocity_to()
       ta.shift_spectrum_to(redshift=, radial_velocity=)

       ta.rest_value =
       ta.doppler_rest = 

       # no setters for these
       ta.redshift
       ta.radial_velocity
```


## Loading Modules
We start by loading the modules we will use for the data reduction. 

For display purposes, we use the static (non-interactive) `matplotlib` backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on `jupyter` lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for working with the data.
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.log import init_logging
from astropy import units as u
from dysh.spectra.spectrum import Spectrum

# These modules are used for file I/O
from dysh.util.files import dysh_data
from pathlib import Path

## Setup
We start the dysh logging, so we get more information about what is happening.
This is only needed if working on a notebook.
If using the CLI through the ``dysh`` command, then logging is setup for you.

In [ ]:
init_logging(2)

# also create a local "output" directory where temporary notebook files can be stored.
output_dir = Path.cwd() / "output"
output_dir.mkdir(exist_ok=True)

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
filename = dysh_data(test="getps")

## Data Loading

In [ ]:
sdfits = GBTFITSLoad(filename)
sdfits.summary()

## Data Reduction

Next we fetch and calibrate the position switched data. We will use this data to show how to change rest frames and Doppler conventions. All of these occur at the spectrum level, so we need to get the spectrum first.

In [ ]:
psscan = sdfits.getps(scan=51, ifnum=0, plnum=0, fdnum=0)

Create the time-averaged spectrum.

In [ ]:
ta = psscan.timeaverage()

## Changing the x-axis of a `Spectrum` Plot
Note this changes the axis of the plot but does not affect the underlying Spectrum object.


### Default Rest Frame

The default plot uses the frequency frame and Doppler convention found in the SDFITS file.
In this case, that is topocentric frame (ITRS) and the optical convention. In the SDFITS file you would find the keyword VELDEF to have the value OPTI-LSR:  (PJT: LSR ?)
```
    print(sdfits['VELDEF'][0])
```

4410.07   topo based radial velocity of target w.r.t. observatory
4386 = VELOCITY   (catalog)  LSR
'RVSYS': 4376,523.966595529,      vsource - vtelescop
'VFRAME': 22,609.232181632025,    rv of velocity frame

In [ ]:
print(sdfits['VELDEF'][0])

In [ ]:
ta

In [ ]:
ta.meta

In [ ]:
ta.meta["VELOCITY"] = 1000000.0
# ta.meta["VELDEF"] = "foobar"
ta.velocity
ta.radial_velocity   # this should be in plot ???    tied to VELDEF

In [ ]:
ta.plot()

For the remainder of the notebook, we focus on the small section near 1.39 GHz where a random spike can be seen. After some trial and error this happens in channel 21921 where the peak value is 0.59789101 K.

explain more why

Lets plot this zoomed spectrum in km/s

In [ ]:
if ta.nchan > 1000:
    print("Taking a zoomed spectrum")
    print(ta[21920:21923])   # print values around the little spike
    ta = ta[21850:22000]     # these are the 150 channels we will zoom into
else:
    print(f"Already had the zoomed spectrum of {ta.nchan} channels")
    print(ta[70:73])
ta.plot(xaxis_unit='km/s')
print(ta.velocity_axis_to("km/s")[71])     # 6256.475    velocity of the spike in this frame 

## Writing a spectrum 

The default `Spectrum.write()` uses its native units (Hz) to write a spectrum. More useful is the spectral axis in km/s. One way is to make a copy of the spectrum while changing the units, using 
[Spectrum.with_spectral_axis_unit()](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum.html#specutils.Spectrum.with_spectral_axis_unit)

In [ ]:
# should not use private functions.   See issue #299 and #527
# this also changed the spectral axis

#ta._spectral_axis = ta._spectral_axis.to("km/s") 
#ta.write("junk1v.tab",format="ascii.commented_header", overwrite=True)

In [ ]:
ta.with_spectral_axis_unit?

In [ ]:
# note cannot change frame
ta.set_frame("gcrs")
ta1 = ta.with_spectral_axis_unit("km/s",velocity_convention="radio", rest_value=1420*u.MHz)
#ta1.plot(xaxis_unit='km/s')
ta1

In [ ]:
ta1.write("ngc5291_spike.tab",format="ascii.commented_header", overwrite=True)

In [ ]:
ta1

In [ ]:
# short cut to prevent something like ta1 to become
ta.with_frame("gcrs").with_spectral_axis_unit("km/s",velocity_convention="radio", rest_value=1420*u.MHz).write("ngc5291_spike.tab",format="ascii.commented_header", overwrite=True)

PJT

radial_velocity=4410.070039086893 km / s
      redshift=0.014820217767934851
doppler_rest=1420405000.0 Hz
    doppler_convention=optical)
    
      but 4410.070039/c = 0.0147104
      
and 0.014820217767934851*c = 4442.99


ok, see above

### Change Rest Frame
You can change the velocity frame by supplying one of the [built-in astropy coordinate frames](https://docs.astropy.org/en/stable/coordinates/index.html#built-in-frame-classes). These are specified by string name.

For example, to plot in the barycentric frame use `"icrs"`. The change from topocentric to barycantric is small, a shift of 22.8 kHz (PJT:  25 km/s or 120 kHz???)

In [ ]:
ta.plot(vel_frame='icrs', xaxis_unit='km/s',grid=True)
print(ta.velocity_axis_to("km/s")[71])     # only in plot, the printed value is stil the TOPO centric

In [ ]:
# PJT:   and then bary is not recognized, where topo is... feature request?
# a.plot(vel_frame='bary', xaxis_unit='km/s')

In addition to the `astropy` frame names, we also allow `'topo'` and `'topocentric'`.

In [ ]:
ta.plot(vel_frame='topo', xaxis_unit='km/s')

### Doppler Convention 
One can also change the Doppler convention between `radio`, `optical`, and `relativistic`.
Here we also change the x-axis to velocity units and use LSRK frame.

PJT: Note astropy claims it's "LSRK", but only "lsrk" works.       

LSRK is the class name, lsrk the astropy names it recognized

In [ ]:
ta.plot(vel_frame='lsrk', doppler_convention='radio', xaxis_unit='km/s')
#ta.plot(vel_frame=LSRK, doppler_convention='radio', xaxis_unit='km/s')   # need th class name imported

Finally, if you plot velocity units on the x-axis with no `vel_frame` given, it will default to the 
frame decoded from the VELDEF keyword in the header if present.

In [ ]:
ta.plot(xaxis_unit="km/s")

## Changing the Spectral Axis of the `Spectrum`.
There are two ways to accomplish this.  One returns a copy of the original Spectrum with the new spectral axis; 
the other changes the spectral axis in place.

The `with_frame()` and `with_velocity_convention()` are used when returning a copy

The `set_frame()` and `set_convention()` are used in place  [PJT: notice the asymmetric used of function names]  -> feature requesy

### A.   Return a copy of the spectrum using [`Spectrum.with_frame()`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.with_frame)

In [ ]:
newspec = ta.with_frame('galactocentric')                              # OPTI-GAL, but the V value has not changed !!!
newspec.plot(xaxis_unit="km/s",doppler_convention='radio')
print(f"The new spectral axis frame is {newspec.velocity_frame}")

#
newspec.write("junk2.tab",format="ascii.commented_header", overwrite=True)
# 1392090698.772761 0.598990
# 1391368046.6171937 0.597891008466955 
print(newspec.velocity_axis_to("km/s")[71]) # 6097.621

In [ ]:
newspec2 = ta.with_frame('galactocentric').with_velocity_convention("radio")
#newspec2 = ta.with_velocity_convention("radio").with_frame('galactocentric')
newspec2.plot(xaxis_unit="km/s")
print(newspec2.velocity_axis_to("km/s")[71]) # 5976.071

One can see that the spectral axis of the new spectrum is different. About 722 kHz.

In [ ]:
ta.spectral_axis - newspec.spectral_axis


### B.   Change the spectral axis in place using 
[Spectrum.set_frame()](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.set_frame)

PJT:   should use the same example as previous copy version

In [ ]:
print(ta.velocity_axis_to("km/s")[71]) # 6256.475
sa = ta.spectral_axis
ta.set_frame('gcrs')
print(f"Changed spectral axis frame to  {ta.velocity_frame}")
print(ta.velocity_axis_to("km/s")[71]) # 6256.365

In [ ]:
ta

In [ ]:
ta.plot(xaxis_unit="km/s")

Here the original the spectral axis has changed, albeit by a very small change.

500 Hz or 0.10 km/s

In [ ]:
ta.spectral_axis - sa

In [ ]:
ta.set_convention('radio')

In [ ]:
ta.plot(xaxis_unit="km/s")
print(ta.velocity_axis_to("km/s")[71]) # 6128.47

In [ ]:
ta.target

In [ ]:
ta.observer

## Other useful functions

### Spectral Axis Conversion

Convert the spectral axis to any units, frame, and convention with [`Spectrum.velocity_axis_to`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.velocity_axis_to). dysh understands some common synonyms like 'heliocentric' for astropy's 'hcrs'.

This returns an array, does not modify the spectrum meta data

? PJT:   why is toframe=   and not  vel_frame=

HeliocentricTrueEcliptic didn't work, but astropy claims it's a frame;   we convert it to lower case?



In [ ]:
ta.velocity_axis_to(unit="pc/Myr", toframe='heliocentric', doppler_convention='radio')

In [ ]:
# ta.velocity_axis_to(toframe='heliocentricTrueEcliptic', doppler_convention='radio')

In [ ]:
ta.velocity_axis_to(unit="km/s")

In [ ]:
ta.velocity_axis_to(unit="GHz")

In [ ]:
ta[70:73]

In [ ]:
ta[70:73].velocity_axis_to(unit="km/s").value[1]

### Spectral Shift

Shift a spectrum in place to a given radial velocity or redshift with 
[Spectrum.shift_spectrum_to()](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D.shift_spectrum_to)

In [ ]:
print(ta.velocity_axis_to("km/s")[71])

In [ ]:
print(f"before shift {ta.spectral_axis}")
#ta.shift_spectrum_to(radial_velocity=0*u.km/u.s)
#ta.shift_spectrum_to(radial_velocity=4410.070039086893*u.km/u.s)
ta.shift_spectrum_to(radial_velocity=2000*u.km/u.s)
print(f"after shift {ta.spectral_axis}")

In [ ]:
ta

Changing the redshift?
Here's some confusion, printing the spectrum it says:

doppler_rest  which is spectrum.rest_value   (say 1420 MHz)


PJT  roundoff I guess
old: 4410.070039086893  0.014820217767934851
new: 4410.070039086889  0.014820217767934851




In [ ]:
print(ta.velocity_axis_to("km/s")[71])  # 6128.575765276228 

In [ ]:
ta.plot(xaxis_unit="km/s")

In [ ]:
ta

In [ ]:
# going back to the original --- fails with target/observer
ta.set_frame('itrs')   # itrs   fails!!     icrs works
ta.set_convention('optical')
ta.plot(xaxis_unit="km/s")
print(ta.velocity_axis_to("km/s")[71]) 

In [ ]:
ta2 = Spectrum.fake_spectrum()
print(ta.observer)

In [ ]:
# Spectrum.fake_spectrum().set_redshift_to(0.1)
# Spectrum.fake_spectrum().set_radial_velocity_to( 4000 * u.km/u.s)

# ValueError: Cannot specify radial velocity or redshift if both target and observer are specified

### Spectral Axis in Wavelengths

The default is angstrom, use [Quantity.to](https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to) to convert to other units.

In [ ]:
ta.wavelength.to('cm')

### Plot in Channels

Also, you can plot the x-axis in channel units

In [ ]:
ta.plot(xaxis_unit='chan')

## Final Stats

Finally, at the end we compute some statistics over a spectrum, merely as a checksum if the notebook is reproducable.

In [ ]:
ta.stats() # rms 0.09109935 K on full, and 0.07586939 K on the 21850:22000 portion of the original

In [ ]:
ta.nchan

In [ ]:
1000 * u.Unit("km/s")

